In [1]:
import pandas as pd
import os
import shutil

In [2]:
attributeDF = pd.read_csv('tablelamps_basematerial.csv')
allDF = pd.read_csv('alltablelamps.csv')

In [3]:
#we want to keep only the attributes that we also have an image of

#in these two cells I want to get the IDs of items in each of the folders (product + cropped)
images_path = 'TableLampsProductImages'
image_extensions = ['.jpg', '.png', '.jpeg']
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path)
         for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
productIDs = [image.split('.')[0].split('/')[-1] for image in images]
print(len(productIDs))

9998


In [4]:
print(len(set(productIDs)))

4290


In [5]:
croppedImages = []
for dp, dn, filenames in os.walk('TableLampsCropped'):
    for f in filenames:
        croppedImages.append(f)
croppedIDs = [image.split('.')[0] for image in croppedImages]
print(len(croppedIDs))

839


In [6]:
#we combine the product and cropped ids and remove duplicates so we have a baseline for what we need
allIDs = croppedIDs + productIDs
allIDs = list(set(allIDs))
allIDs = [int(item) for item in allIDs if item != '']

In [7]:
len(allIDs)

4301

In [8]:
print(allDF.loc[allDF['OMS_SKU'].isin(allIDs)].shape)
print(attributeDF.loc[attributeDF['omsid'].isin(allIDs)].shape)

(10151, 11)
(4198, 3)


In [9]:
#the finalID are all of the ids of items in the attribute csv that we have images of
finalattributeDF = attributeDF.loc[attributeDF['omsid'].isin(allIDs)]
finalIDs = []
for i,row in enumerate(finalattributeDF.itertuples()):
    finalIDs.append(row.omsid)
len(finalIDs)

4198

In [10]:
#we then use finalIDs to filter the original DF.
allDF['cc'] = allDF.groupby(['OMS_SKU']).cumcount()
allDF = allDF.loc[allDF['cc']==0]
allDF = allDF.drop(['cc'],axis=1)
allDF.shape

(4376, 11)

In [11]:
#export to redownload the product images for basecolor
allDF = allDF.loc[allDF['OMS_SKU'].isin(finalIDs)]
allDF.to_csv('TablelampRedownloadBaseMaterial.csv',index=False)

In [12]:
allDF.shape

(4175, 11)

In [13]:
#After redownloading the images, we create ANOTHER finalIDs list because there were some ids that were not included
images_path = 'TableLampsBaseMaterialProductImages'
image_extensions = ['.jpg', '.png', '.jpeg']
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path)
         for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
productIDs = [image.split('.')[0].split('/')[-1] for image in images]
print(len(productIDs))

4175


In [14]:
allIDs = croppedIDs + productIDs
allIDs = list(set(allIDs))
allIDs = [int(item) for item in allIDs if item != '']
print(len(allIDs))

4179


In [15]:
#verify this
attributeDF.loc[attributeDF['omsid'].isin(allIDs)].shape

(4179, 3)

In [137]:
#lastly we want only the cropped images that also have an attribute
finalatDF = attributeDF.loc[attributeDF['omsid'].isin(allIDs)]
finalIDs = []
for i,row in enumerate(finalatDF.itertuples()):
    finalIDs.append(row.omsid)
len(finalIDs)

3766

In [142]:
type(finalIDs[0])

int

In [16]:
#want the cropped IDs that are in the finalIDs
print(len(croppedIDs))
croppedIDs = [int(ID) for ID in croppedIDs if ID != '']
print(len(croppedIDs))
newcroppedIDs = [ID for ID in croppedIDs if ID in finalIDs]
print(len(newcroppedIDs))

839
838
838


In [148]:
#with newcroppedIDS you want to keep all of the cropped images that are in this list
ls = []
newcroppedpath = 'TableLampsShadeShapeCropped'
for dp, dn, filenames in os.walk('TableLampsCropped'):
    for f in filenames:
        if (f.split('.')[0]) != '':
            if int(f.split('.')[0]) in newcroppedIDs:
                ls.append(os.path.join(dp,f))
print(len(ls))
print(ls[0])
if not os.path.isdir(newcroppedpath):
    os.makedirs(newcroppedpath)

for i,item in enumerate(ls):
    shutil.copy(item, newcroppedpath + '/' + item.split('/')[-1])

816
TableLampsCropped/304030807.jpg


In [156]:
#verify one last time
productIDs = [int(item) for item in productIDs]
allIDs = newcroppedIDs + productIDs
allIDs = list(set(allIDs))
allIDs = [int(item) for item in allIDs if item != '']
print(len(allIDs))

3766


In [17]:
attributeDF.loc[attributeDF['omsid'].isin(allIDs)].to_csv('finaltablelamps_basematerial.csv',index=False)

In [157]:
attributeDF.loc[attributeDF['omsid'].isin(allIDs)].shape

(3766, 3)